In [73]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.ops import resources
from tensorflow.contrib.tensor_forest.python import tensor_forest

# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import pandas as pd
import numpy as np

In [74]:
# Load data
train = pd.read_csv('../data/datosEntrenamiento.csv', header = None)
test = pd.read_csv('../data/datosPrueba.csv', header = None)
validation = pd.read_csv('../data/datosValidacion.csv', header = None)
train=train.rename(columns = {36:'class'})
test=test.rename(columns = {36:'class'})
train_class_dummies = pd.get_dummies(list(train['class']))
train_class_dummies.columns = ['class1','class2','class3','class4','class5','class6']
test_class_dummies = pd.get_dummies(list(test['class']))
test_class_dummies.columns = ['class1','class2','class3','class4','class5','class6']
train = pd.concat([train,train_class_dummies], axis = 1)
test = pd.concat([test,test_class_dummies], axis = 1)
x_test = test.drop(test.columns[list(range(36,43))], axis = 1).values
y_test = test.drop(test.columns[list(range(36)) + list(range(37,43))],axis = 1).values
x_train = train.drop(train.columns[list(range(36,43))], axis = 1).values
y_train = train.drop(test.columns[list(range(36)) + list(range(37,43))],axis = 1).values

In [75]:
# Parameters
num_steps = 500 # Total steps to train
batch_size = 100 # The number of samples per batch
num_classes = 6 
num_features = 36 # Each image is 28x28 pixels
num_trees = 10
max_nodes = 1000

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_features])
Y = tf.placeholder(tf.float32, [None, num_classes])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

In [76]:
# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
    resources.initialize_resources(resources.shared_resources()))

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_output_columns': 7, 'feature_bagging_fraction': 1.0, 'valid_leaf_threshold': 1, 'checkpoint_stats': False, 'initialize_average_splits': False, 'pruning_type': 0, 'prune_every_samples': 0, 'dominate_fraction': 0.99, 'max_fertile_nodes': 0, 'early_finish_check_every_samples': 0, 'dominate_method': 'bootstrap', 'bagging_fraction': 1.0, 'regression': False, 'param_file': None, 'bagged_num_features': 36, 'use_running_stats_method': False, 'max_nodes': 1000, 'split_finish_name': 'basic', 'leaf_model_type': 0, 'stats_model_type': 0, 'bagged_features': None, 'num_features': 36, 'split_after_samples': 250, 'num_outputs': 1, 'collate_examples': False, 'split_type': 0, 'num_classes': 6, 'num_splits_to_consider': 10, 'split_name': 'less_or_equal', 'finish_type': 0, 'inference_tree_paths': False, 'split_pruning_name': 'none', 'base_random_seed': 0, 'num_trees': 10, 'model_name': 'all_dense'}


In [77]:
# Start TensorFlow session
sess = tf.train.MonitoredSession()

# Run the initializer
sess.run(init_vars)

# Training
for i in range(1, num_steps + 1):
    batch_x = x_train
    batch_y = y_train
    _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

# Test Model
test_x, test_y = x_test, y_test
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


ValueError: Cannot feed value of shape (4504, 1) for Tensor u'Placeholder_1:0', which has shape '(?, 6)'